#### Spotify API:
- API connection
- Data sorting
- Data to CSV

In [32]:
# API connection

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


CLIENT_ID = '3f8cc110d1534d4ba1cd6d063cf114e3'
CLIENT_SECRET = '2c63ffda0e7d4247aaa719de974460bf'


auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [33]:
# Data sorting

spotify_data = []

def data_extraction_s(list_url):

    for url in list_url:

        playlist = sp.playlist(url, fields=None, market=None, additional_types= 'track')

        
        for item in playlist["tracks"]["items"]:
            track = item["track"]
            artist = item["track"]["artists"]
            eurovisionyear = playlist['name']

            spotify_data.append({
                "Artist": artist[0]["name"].title(),
                "Track": track["name"].title(),
                "Spotify_Duration": track["duration_ms"],
                "Spotify_EurovisionYear" : eurovisionyear.title(),
                "Spotify_Popularity": track["popularity"]
                })
        
        

In [34]:
# Data to CSV

raw_data_s = data_extraction_s([
    "https://open.spotify.com/playlist/3ZdQUt8Tmtt7oOU8UM2koe?si=eea27348cf654d21", 
    "https://open.spotify.com/playlist/5h0sQpJnLVzgy5iOH1UNcl?si=3a1d926528484506", 
    "https://open.spotify.com/playlist/3EBtzzGyrlDUr1QvXVGz0h?si=DMNkLRIjQE6THtHUYc-oQw",
    "https://open.spotify.com/playlist/61fyG82H3Cha91cgKgQT6S?si=o-Xs3DfHRZyRuR8qaY9mxAW",
    "https://open.spotify.com/playlist/2HMW5JNZNh9HLfIHNzY0DP?si=f29ed55294394682"
    ])

df_spotify = pd.DataFrame(spotify_data)
df_spotify

df_spotify.to_csv('Spotify_data.csv')


In [35]:
df_spotify

,Artist,Track,Spotify_Duration,Spotify_EurovisionYear,Spotify_Popularity
0,Chingiz,Truth - Radio Edit,209411,Eurovision 2019,0
1,John Lundvik,Too Late For Love,178133,Eurovision 2019,0
2,Kate Miller-Heidke,Zero Gravity,177443,Eurovision 2019,38
3,Michela,Chameleon,179698,Eurovision 2019,0
4,Mahmood,Soldi,195476,Eurovision 2019,0
...,...,...,...,...,...
205,Fahree,Özünlə Apar,182090,Eurovision 2024,40
206,Sarah Bonnici,Loop,179727,Eurovision 2024,42
207,Megara,11:11,180087,Eurovision 2024,39
208,Natalia Barbu,In The Middle,178853,Eurovision 2024,37


#### LastFM API: INGLEEEEEEESSSSSSSSSS!!!!!!!


- Artists
- Track

In [36]:
import pandas as pd
import requests
import json
import re  

In [ ]:
most_popular_from_spotify = ["Chanel", "Loreen", "Måneskin", "Sam Ryder", "Käärijä"]

#### Eurovision API:
- Api connection & Sorting
- Data to CSV

In [5]:
import requests
import pandas as pd
import json

In [6]:
# API connection & Sorting

eurovision_data = []
def data_extraction_e(list_years):

    eu_data = requests.get(f"https://eurovisionapi.runasp.net/api/contests/{list_years}/")

    eu_data_json = eu_data.json()
    
    if 'contestants' in eu_data_json:
        
        for idx, i in enumerate(eu_data_json["contestants"], start=1):

            contestant_name = i["artist"]
            song_name = i["song"]
            country = i["country"]

            eurovision_data.append({
                "Ranking": idx,
                "Artist": contestant_name.title(),
                "Track": song_name.title(),
                "Country": country.upper(),
                "Eurovision_Year": list_years
                })

In [7]:
# Data to CSV

eurovision_years= [2019,2021,2022,2023,2024]

for year in eurovision_years:
    raw_data_e = data_extraction_e(year)
df_eurovision = pd.DataFrame(eurovision_data)
df_eurovision

df_eurovision.to_csv('Eurovision_data.csv', index=False)


#### MySQL Data Base
- MySQL connection
- Schema Creation and Use
- Tables Creation

In [ ]:
# MySQL connection

import mysql.connector

try:
    cnx = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1')
    

    mycursor = cnx.cursor()
    

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

In [19]:
# Schema Creation and Use

query_create_schema = """CREATE SCHEMA StreamMusic"""
mycursor.execute(query_create_schema)
mycursor.execute('USE StreamMusic')

In [ ]:
# Tables creation


the_tables = {  'API_Eurovision' :          """CREATE TABLE api_eurovision (
                                                Ranking INT NOT NULL,
                                                Artist VARCHAR(45) NOT NULL,                              
                                                Track VARCHAR(70) NOT NULL,
                                                Country VARCHAR(3) NOT NULL,
                                                Eurovision_Year YEAR NOT NULL)""",
                                                
                'API_Spotify_Eurovision' :    """CREATE TABLE api_spotify_merged_eurovision (
                                                Artist VARCHAR(45),
                                                Track VARCHAR(70),
                                                Spotify_Duration INT,
                                                Spotify_EurovisionYear VARCHAR(25),
                                                Spotify_Popularity VARCHAR(10),
                                                Eurovision_Ranking INT,
                                                Eurovision_Country CHAR(2),
                                                PRIMARY KEY (Artist, Track)
                                                )""", 
                
                'API_LastFM_Info_Artists' :  """CREATE TABLE api_lastfm_info_artists (
                                                Artist VARCHAR(45),
                                                Listeners INT,
                                                Playcount INT,
                                                Genre1 VARCHAR(45),
                                                Genre2 VARCHAR(45),
                                                Genre3 VARCHAR(45),
                                                Similar_Artists1 VARCHAR(45),
                                                Similar_Artists2 VARCHAR(45),
                                                Similar_Artists3 VARCHAR(45),
                                                PRIMARY KEY (Artist),
                                                CONSTRAINT fk_artists_from_s
                                                        FOREIGN KEY (Artist)
                                                        REFERENCES  api_spotify_merged_eurovision (Artist)
                                                        ON UPDATE CASCADE
                                                        ON DELETE CASCADE
                                                )""",
                
                'API_LastFM_Top_Tracks' :    """CREATE TABLE api_lastfm_top_tracks (
                                                Artist VARCHAR(70) NOT NULL,
                                                Release_year YEAR NOT NULL,
                                                Top_track VARCHAR(150),
                                                PRIMARY KEY (Artist),
                                                CONSTRAINT fk_tracks_from_s
                                                        FOREIGN KEY (Artist)
                                                        REFERENCES  api_spotify_merged_eurovision (Artist)
                                                        ON UPDATE CASCADE
                                                        ON DELETE CASCADE
                                                )""" 
                }



for k, v in the_tables.items():
        print(f'Table created: {k}')
        mycursor.execute(v)

        print('------------')



Table created: API_Eurovision
------------
Table created: API_Spotify_Eurovision
------------
Table created: API_LastFM_Info_Artists
------------
Table created: API_LastFM_Top_Tracks
------------


#### Data Import

- Method
- Queries
- Data Import

In [11]:
# Data Import Method

def data_import(query,values):

    mycursor = cnx.cursor()
    mycursor.executemany(query, values)
    print(mycursor.rowcount,'values entered')
    cnx.commit()
 

In [ ]:
# Data Import Queries 
query_e_import =            """ INSERT INTO API_EUROVISION (Ranking, Artist, Track, Country, EurovisionYear)
                                VALUES (%s, %s, %s, %s, %s) """

query_s_e_import =            """ INSERT INTO api_spotify_merged_eurovision (Artist, Track, Spotify_Duration, Spotify_EurovisionYear, Spotify_Popularity,)
                                VALUES (%s, %s, %s, %s, %s) """

query_l_artist_import =     """ INSERT INTO api_lastfm_info_artists (Artist, Listeners, Playcount, Genre1, Genre2, Genre3, Similar_Artists1, Similar_Artists2, Similar_Artists3)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) """

query_l_toptracks_import =  """ INSERT INTO api_lastfm_top_tracks (Release_year, Artist, Top_track)
                                VALUES (%s, %s, %s) """



In [25]:
# Eurovision Data Import

e_file = pd.read_csv('Eurovision_data.csv')

e_values = list(e_file.itertuples(index=False, name=None))
eurovisionimp = data_import(query_e_import, e_values)

194 values entered


In [26]:
# Spotify Data Import

s_file = pd.read_csv('Spotify_data.csv',index_col=0)

s_values = list(s_file.itertuples(index = False, name=None))
spotifyimp = data_import(query_s_e_import, s_values) 

208 values entered


In [ ]:
# LastFM Artists Import

# l_file = pd.read_csv(,index_col=0)

